## 3445. 奇偶频次间的最大差值 II

https://leetcode.cn/problems/maximum-difference-between-even-and-odd-frequency-ii/

In [39]:
# 复杂度：O(N^2)，依然超时
class Solution:
    def maxDifference(self, s: str, k: int) -> int:
        assert len(s) >= 3, "too short string: s"
        result = None
        k = max(k, 3)  # 最少为3，否则无结果
        self.cache = {}  # 计算结果缓存，多级缓存，{key-起始位置: {key-子串长度： {'key-字符': num-字符个数}}}

        result = None  # 根据result可以推断至少子串的长度是result+4，如-1只有在长度不小于3的子串中出现，1只能在长度不小于5的子串中出现。result比为奇数
            
        for i in range(0, len(s) - k + 1):  # 子串的起始位置范围
            j = k if result is None else max(k, result + 4 + 1)  # 如果已经在长度为result + 4的子串中获得过理论最大值，只有可能在更长的子串中提升

            archor_start = self.cache.get(i-1)  # 获取上一行的临近节点结果作为锚点，以加速计算；
            archor = None if archor_start is None else archor_start.get(j)
            if archor is not None:
                archor = archor.copy()
                if archor[s[i-1]] == 1:
                    del archor[s[i-1]]
                else:
                    archor[s[i-1]] -= 1

                del self.cache[i - 1] # 上一行的缓存已经没必要了，调试的时候可以查看这个cache

            while j <= len(s) - i:
                if archor is None:
                    self.update_cache(i, j, self.char_count(s[i:i+j]))
                else:
                    archor = archor.copy()
                    archor[s[i+j-1]] = archor.get(s[i+j-1], 0) + 1
                    self.update_cache(i, j, archor)
                
                max_diff = self.max_diff(self.cache[i][j])

                if (result is None and max_diff is not None) or (max_diff is not None and max_diff > result):
                    result = max_diff
                    
                j += 1
            
            if result is not None and i >= len(s) - (result + 4):  # 当起始位置的子串已经不可能长过这个理论最短子串时，就不用继续了
                break
        
        return result
    
    def char_count(self, s):

        counter = {}
        for e in s:
            counter[e] = counter.get(e, 0) + 1
        
        return counter
    
    def update_cache(self, i, j, char_count_cache):
        if i not in self.cache: self.cache[i] = {}
        self.cache[i][j] = char_count_cache

    def max_diff(self, char_dict):
        max_odd = None
        min_even = None
        for v in char_dict.values():
            if v & 1 == 1:
                max_odd = max(max_odd, v) if max_odd is not None else v
            else:
                min_even = min(min_even, v) if min_even is not None else v
        return max_odd - min_even if max_odd is not None and min_even is not None else None


In [25]:
list(range(0, 5, 2))

[0, 2, 4]

In [ ]:
# 复杂度：O(N^2)，依然超时
# 通过一次遍历正反的方式，去掉缓存的复制操作
class Solution:
    def maxDifference(self, s: str, k: int) -> int:
        assert len(s) >= 3, "too short string: s"
        result = None
        k = max(k, 3)  # 最少为3，否则无结果
        cache = None  # 计算结果缓存，{'key-字符': num-字符个数}

        result = None  # 根据result可以推断至少子串的长度是result+4，如-1只有在长度不小于3的子串中出现，1只能在长度不小于5的子串中出现。result必为奇数
            
        for i in range(0, len(s) - k + 1, 2):  # 子串的起始位置范围
            min_length = k if result is None else max(k, result + 5)  # 如果已经在长度为result + 4的子串中获得过理论最大值，只有可能在更长的子串中提升
            max_length = len(s) - i
            j = min_length
            reverse = False

            while j >= min_length and j <= max_length:
                if cache is None:
                    cache = self.char_count(s[i:i+j])
                    max_diff = self.max_diff(cache)
                else:
                    if not reverse:
                        cache[s[i+j-1]] = cache.get(s[i+j-1], 0) + 1
                        max_diff = self.max_diff(cache)
                        if j == max_length:  # 掉头
                            reverse = True
                            cache[s[i]] = cache[s[i]] - 1
                            if cache[s[i]] == 0: del cache[s[i]]
                            i += 1
                            j -= 1
                            max_length -= 1
                            if max_length >= min_length:
                                max_diff_2 = self.max_diff(cache)
                                max_diff = max(max_diff, max_diff_2) if max_diff is not None and max_diff_2 is not None else max_diff if max_diff_2 is None else max_diff_2
                        
                    else:
                        cache[s[i+j]] -= 1
                        if cache[s[i+j]] == 0: del cache[s[i+j]]
                        max_diff = self.max_diff(cache)
                        if j == min_length:
                            cache[s[i]] = cache[s[i]] - 1
                            if cache[s[i]] == 0: del cache[s[i]]  # 已经比min_length还小了，所以本次不计算max_diff

                if (result is None and max_diff is not None) or (max_diff is not None and max_diff > result):
                    result = max_diff
                    min_length = max(min_length, result + 5)
                
                if reverse and j == min_length:
                    break

                j = j+1 if not reverse else j-1
            
            if result is not None and i >= len(s) - (result + 4):  # 当起始位置的子串已经不可能长过这个理论最短子串时，就不用继续了
                break
        
        return result
    
    def char_count(self, s):

        counter = {}
        for e in s:
            counter[e] = counter.get(e, 0) + 1
        
        return counter

    def max_diff(self, char_dict):
        max_odd = None
        min_even = None
        for v in char_dict.values():
            if v & 1 == 1:
                max_odd = max(max_odd, v) if max_odd is not None else v
            else:
                min_even = min(min_even, v) if min_even is not None else v
        return max_odd - min_even if max_odd is not None and min_even is not None else None


In [43]:
solution = Solution()

In [37]:
assert solution.maxDifference(s = "330130303114333", k=13) == -3

In [44]:
solution.maxDifference(s="114314001311021414104300302211123341300202434414142220244314042142240113321300014420004221401143031432202414401323030321323213111000200440234313413212441232202213232111312004121220433043112333033221131103202003420303103004102434200400024001311120203010300121104230202322342133023114034214143032032044314204203423332211232404012023332330444413040441003301212434034002243212040021104441301303300101422342100114441324234403040441330244401242140131200021242231011234214433341130400241010324121211224304321040012320013113114040143133413444204033320421123440213132031231023232334234122143431012342212132240012320112034331124031442034012042033324311440233414321101230444313222413020022421042112401420330044332124103334332244042013021012144103102414303032223203044043144321444124402411121103303332220113400043233110032212434432133024132403441200030111030323403143222110010423304132044423201030034424401034302014333022044430131023323420411311110212410330202112132002243101420442334431034222100412344240444004143013203020104430432310022421241421003010411444400233243144233010304014032001023304010024401310413233224210310241231201012242300212020240003430124320430110340120431214142321043330124332001201342100413340011233044424144222044342040201340033321210214144421430002313422410022222430011114423000044422333000134302032221310100321302342414343031342423342023244340214330342404423044233430340034233030033222403320043013001224020440020214334322344034202434130130303001403304433243300404222233334021120012414121424012440301230134433012102004204042142311102312144012133022004432014230313400412322042343131420240022130202014234223303113044413103003042201232320410443314324410344210213322023122420144204312121300141144323002031231010314312433101102444303232020022413043401314012133334124112231433322100324103114122310014330031230243022304210422334222324104212444011211441140224124433300330240232144201221344201221133013142434104234400442220141434230200431033013004031441011441323040242334211244004144200023122211420231021213304042222121123014131034304132303121334442233331200442422013104331421143240210214033444010241210404011320201041234420034234010214121313304324322243312132113303341202113020304124230411144233411122323332301110004310303211120044220210103210401443312311130232400240301024310014320001230223044230134213321004223311313210322131101122013211043230123430031304222134404401101301342104020012213040222244204321013211221434424204314434031200123201330133444302004411130214031214424412400233100343323323340132233423341011414010400011123002240302111132021321402010213110314314244241141412400210423411320402143244141302201231142432431113240434424311030222312010331311313314420444413321120002323323003422301100230041310404333134201033040320100003230444121134124304440421242204040414130134344130032424314342403220442142134241324342041301114113311233202244034334403342340302440344040444203023232323412100233140124234222013220210232010430213321331212410343242241313234320031003420424121313233444220304402130213042120313003401020232223001234104410401240314330314213111142402410231040210233404030202112340202424031024301030204341120213344114214124424303011020443243322113241010122113232322304301412240020431223102400014202324432040231102432242121023141230124431101202223411410200200413012203221411130434304300142111140233114422322202433312430340334233213112403031004032022410320214041021022203423301102233113002112430341123313242211222010214403300042120311113113034432443034314411320412400124404432343231314023122000422303012044304143214110101203343320310223222233021421243141300301304140344320013220003242112313022144130021202343044200422321214322234130420234344213333332043023303301204010012233104240442223230040214134223244410444430233021004232124332320324212212343021433000111334311233444020130023242310233004344043121124033242203024123031021101101101411431211421341002303104014042232004344130404430433020142034040213220434144102243134003132110102214220010320024012304224112214000410321334434323143044220003211423240402140424040004402414023430322441414124414403032443202331240220330134300400134204201424121224413224224320021101443040144424011220243014143134441031121221223102442001421342441230123004313120414340344002144130444040203311442443104341221210221413400224423424224012043121340413401432344323304040014233404202300002202021132400343214404442043211022032140114203141023334030340022134430341324330044321010033343300331040344313131432133323412413304342200300030013301103211110101232011040014434143041114334313144322121222114444111100144240041023101411313404124111430412043144330221424413043002143104403123430012241144422111200114121434123110112440341411341324323303323024001021220442443142231001001320042331301204233014033140123200030201443440433144310242200324243141101124040102023340204100120104301332003101101303241011010411300014234143043424314323312430243421203011312412210411410422202112111023314340040144433333401223403333124201104324233434022032031120141112422443314443342221342240313320414131113022423130300123014422322202112422312310220032111021423331303144101400424244434330000303232031312200111033231143021201421224341314323212104420233322040121431042032141010321331010032002403011122324332220024311120112030313022041104040212340313110422010204313002340343121330323402430312411224331043224220043401302111300344312230",
                       k=1586)

95

In [45]:
solution.maxDifference(s="1132222333023220022000130412131433204111323413130444111313101130434221301124024001002234214104113430033002343240012424122010130003130124241323131031301330333031424414223441123243333020113302243300303102443243241313024221211413223212313100240020313240400130440101242211001042120010004213401043120122044104300001202303012322233243221040402303120301030414102433142142230041412023303433243144121004023022030320432100130123411103132314042103003430140231212310201241314312202433121213433112100323320300304401444333202102433043400220233123433023124031040202241342014204234312124141400144100220442232331113133203443013010411333404214134330323214210110144342320434444042041202223421000000032000241224230100111322403103323123122202012220140231014234432022231040330143420143034340103032130042423234332223241221400202402103440312221444133120001333033111310421210200143312401202232424042021420140333231241341222314141234321201424340324021323342014434202200123021240430331113434040112203240440031110434140104212034031144304313214334344100401241341424331334022344134243143301140033240021041120220341400444424240043114223443112134203331104300401424340142311332320412110223222141041221020013332412123000434232330012210301404021031313034103204024142203432424442400122142320324002201143210201020002032433112023444234310001311424140302423140300222000220140404303343002022234202243142303303214120303021142411400031204244303434234031432212234200211304002401432220420213103233021341120141020213434320204441240342014413030100014403324320011444022441313041243341304124201030043212321331314102010412404443303322404144444043421404011314111433044332132233214431322030002204413201314233001132301442424212224310134241013442214200403344010223101434310321220122043332432442002134123401140213020031224424333300433134200401023430203222424201132423111010220002304424144400120441003321122312031224311423030001223022004241224142312223113104140333322234413023131032420140020001430331210041321320113434122413212341300012410240120233342031144030031214044230220124241420322043434442043140440044430142444102414304101001203022421144302002243432110211122043104301443410402000022310031410320144244422403002342314010021040420303120342241130100031231010303311123334240211404420204142111133104033304041020331434330414232402211023144400221224331121130033402324421434022322410212002021412211043040201022331330341021323100133213013000314332310443110310322104332203322243324111401003330404010022000123323413113014200432143422110112044023030132400333102214423201434220130231342323240230103134133320320033312114413434111432243021311100111443110001232440441111131331044244142104231130132432243244211331144011244234340313340134332034223221014443104123323333431323231403342234214440301222311114114202030002322130143222342413230301212020443201123233124014120033404143012031222440442301243242324444210230024331033231131130032020232413043300124140120340134101312302443311030024441341021143242142411001122124313434220102310104024333332303033210030044232201433411413141023024343112432322032430033311034221240132410224211003143440023230204000311330300221100141100432331020113212303031004200122130410432340212340331332010410241410122440134442332444133433212132001324420404204030114232123204310130402314103311110114011043132114131141444144133220432401230404131032343302434433121240121142323032130044212212311343124341420303031411324144424132120201130142432203031341404400430444342212011442143044142433034420142040230322024302422041034134032013123440412123033241210302021243230032001131141000342131402430313242132101202330014010401201410023000414224010323321311040332321013231131421242320340113041220314144422132310430312031030014340411013230434011223400030422130123002444211341004333034300220214122022424234440001404123421143033021334004323041303242313024210041013004130032112233342113122310230222321141220142321032231004021442100144214343201240404434314320103430420212032333434233432144440011423132230124344002334031442234231033204040402102402043214413034224321032214212011322332133034214300241214140241420142441302431232144133333414023324223433424441431240041124222111214234322041443324401414102134400032410030344121222201413300102434234324201410400000402142132412031232000444221344324422410143000420412034033334001400000302101023440013023004230222322312004104432330131043314003004023444241401303404223213013102311020332044434302130303424113233442124133132234402132223020244341041243342030134001414130110444404222330023114034320043213041003333220430411332314000124404242221404011140422134132021122002042102304134420134220131234222342401340324042422240412322332421033222121024422120111000243421200301341443121112311331233342102110012144401310324403022401103121411212044140111234111344121130314313421224344011122014120130411043103032030132042443230241104032102210240121332311141103004322222442220343044413111032440103213013033211424340100430010111433211323240311212202240341043024004143331300411431033344324013420242312310023021412123210312111033414220434322333321202302030111404243403421320214033000313344402324104104134312421242222222343320243021231021034102200143114010241404343030031331042311030210223303220102323114404334414140103244123324040231141041424100110012030334210444322414222134031411411400221230333234421313230313004002402342211310032111223331343014244133240124021213111311002114140002044221113041140404342114433442444403302131234222302434141102414014442024124102314101324300322111124044243320442320012032004102142300332021022343032324224030341122323322212114014222244443323333114041234001104100210200004333433241020214043132444232140444244433340411220001120023334113221042430122330042124022440332403410221144334020214132040103223033033140212312414401131021202324411132042341142242210241002241002203133320304132024431132431410414042233312324002231100220330022212431130221031424313003013204144232332101204314301431121243030013142144424103344443321303320033030403030330340334130124210013",
                       k=52)

83